# Ch.5 기하학적 변환
## 5.1 이동, 확대/축소, 회전
### 5.1.1 이동
- dst = cv2.warpAffine(src, mtrx, dsize, [, dst, flags, borderMode, borderValue])
- src : 원본영상
- mtrx : 2x3 변환행렬, dtype = float32
- dsize : 결과 이미지 크기
- flags : 보간법 알고리즘 선택 플래그
    - cv2.INTER_LINEAR : 기본 값, 인접한 4개 픽셀 값에 거리 가중치 사용
    - cv2.INTER_NEAREST : 가장 가까운 픽셀 값 사용
    - cv2.INTER_AREA : 픽셀 영역 관계를 이용한 재샘플링
    - cv2.INTER_CUBIC : 인접한 16개의 픽셀값에 거리 가중치 사용
    - cv2.INTER_LANCZ0S4 : 인접한 8개의 픽셀을 이용한 란초의 알고리즘
- borderMode : 외곽 영역 보정 플래그
    - cv2.BORDER_CONSTANT : 고정색상값
    - cv2.BORDER_REPLICATE : 가장 자리 복제
    - cv2.BORDER_WRAP : 반복
    - cv2.BORDER_REFLECT : 반사
- borderValue : cv2.BORDER_CONSTANT의 경우 사용할 색상 값 (default=0)
- dst : 결과 이미지 (numpy 배열)

In [6]:
# 평행이동 (translate)
import cv2
import numpy as np

img = cv2.imread('img/fish.jpg')
rows, cols = img.shape[0:2]
dx, dy = 100, 50

# 변환 행렬 생성
mtrx = np.float32([[1, 0, dx], [0, 1, dy]])

# 단순 이동
dst = cv2.warpAffine(img, mtrx, (cols+dx, rows+dy))

# 탈락된 외곽픽셀을 blue로 보정
dst2 = cv2.warpAffine(img, mtrx, (cols+dx, rows+dy), None, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT, (255, 0, 0))

# 탈락된 외곽픽셀을 반사로 보정
dst3 = cv2.warpAffine(img, mtrx, (cols+dx, rows+dy), None, cv2.INTER_LINEAR, cv2.BORDER_REFLECT)

cv2.imshow('original', img)
cv2.imshow('trans', dst)
cv2.imshow('BORDER_CONSTANT', dst2)
cv2.imshow('BORDER_REFLECT', dst3)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 5.1.2 확대/축소
###  [[x'], [y']] = [[a 0 0], [0 b 0]][[x], [y], [1]]

In [7]:
# 행렬을 이용한 확대와 축소
import cv2
import numpy as np

img = cv2.imread('img/girl.jpg')
height, width = img.shape[:2]

# 0.5 배 축소 변환행렬
m_small = np.float32([[0.5, 0, 0], [0, 0.5, 0]])

# 2배 확대 변환행렬
m_big = np.float32([[2, 0, 0], [0, 2, 0]])

# 보간법 적용 없이 확대/축소
dst1 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)))
dst2 = cv2.warpAffine(img, m_big, (int(height*2), int(width*2)))

# 보간법 적용한 확대/축소
dst3 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)), None, cv2.INTER_AREA)
dst4 = cv2.warpAffine(img, m_big, (int(height*2), int(width*2)), None, cv2.INTER_CUBIC)

cv2.imshow('original', img)
cv2.imshow('small', dst1)
cv2.imshow('big', dst2)
cv2.imshow('small INTER_AREA', dst3)
cv2.imshow('big INTER_CUBIC', dst4)
cv2.waitKey(0)
cv2.destroyAllWindows()

### resize
- dst = cv2.warpAffine(src, dsize, dst, fx, fy, interpolation)
- src : 원본영상
- dsize : 출력 이미지 크기, 생략 시 fx, fy 적용 (width, height)
- fx, fy : 크기 배율, 생략 시 dsize 적용
- interpolation : 보간법 알고리즘 (cv2.warpAffine()과 동일)
- dst : 결과 이미지 (numpy 배열)

In [10]:
# resize
import cv2
import numpy as np

img = cv2.imread('img/fish.jpg')
height, width = img.shape[:2]

# 크기지정으로 축소
dst1 = cv2.resize(img, (int(height*0.5), int(width*0.2)), interpolation=cv2.INTER_AREA)

# 배율 지정으로 확대
dst2 = cv2.resize(img, None, None, 2, 2, cv2.INTER_CUBIC)

cv2.imshow('original', img)
cv2.imshow('small', dst1)
cv2.imshow('big', dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 5.1.3 회전
- x' = x*cos(a) - y*sin(a)
- y' = x*sin(a) + y*cos(a)

- mtrx = cv2.getRotationMatrix2D(center, angle, scale)
    - center : 회전 축 중심 좌표, tuple(x, y)
    - angle : 회전 각도, 60진법
    - scale : 확대/축소 배율

In [12]:
# 변환행렬로 회전
import cv2
import numpy as np

img = cv2.imread('img/fish.jpg')
rows, cols = img.shape[0:2]

# 라디안 각도계산 (60진법 호도법으로 변경)
d45 = 45.0*np.pi/180
d90 = 90.0*np.pi/180

# 회전을 위한 변환행렬 생성
m45 = np.float32([[np.cos(d45), -1*np.sin(d45), rows//2],    # 좌표를 가운데로 옮기기 위한 값
                    [np.sin(d45), np.cos(d45), -1*cols//4]]) # 좌표를 가운데로 옮기기 위한 값
m90 = np.float32([[np.cos(d90), -1*np.sin(d90), rows],
                    [np.sin(d90), np.cos(d90), 0]])

# 회전 변환행렬 적용
r45 = cv2.warpAffine(img, m45, (cols, rows))
r90 = cv2.warpAffine(img, m90, (cols, rows))

cv2.imshow('original', img)
cv2.imshow('45', r45)
cv2.imshow('90', r90)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# 회전 변환행렬
import cv2
import numpy as np

img = cv2.imread('img/fish.jpg')
rows, cols = img.shape[0:2]

# 회전축 : center, angle: 45, scale : 0.5
m45 = cv2.getRotationMatrix2D((cols/2, rows/2), 45.0, 0.5)
# 회전축 : center, angle: 270, scale : 1.5
m270 = cv2.getRotationMatrix2D((cols/2, rows/2), 270.0, 1.5)

# 변환행렬 적용
img45 = cv2.warpAffine(img, m45, (cols, rows))
img270 = cv2.warpAffine(img, m270, (cols, rows))

cv2.imshow('original', img)
cv2.imshow('45', img45)
cv2.imshow('270', img270)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 5.2 뒤틀기
### 5.2.1 Affine 변환 : 이동, 확대/축소, 회전을 포함하는 변환으로 직선, 길이의 비율, 평행성을 보존하는 변환
- 3개의 점을 짝 지어 매핑할 수 있다면, 변환행령 거꾸로 계산 가능
- matrix = cv2.getAffineTransform(pts1, pts2)
    - pts1 : 변환 전 좌표 3개 (numpy 3x2, float32)
    - pts2 : 변환 후 좌표 3개 (numpy 3x2, float32)
    - matrix : 변환 행렬 반환, 2x3

In [18]:
# affine transform 행렬
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('img/fish.jpg')
rows, cols = img.shape[0:2]

# 변환 전, 후 각 3개의 좌표 생성
pts1 = np.float32([[100, 50], [200, 50], [100, 200]])
pts2 = np.float32([[80, 70], [210, 60], [250, 120]])

# 변환 전 좌표 이미지에 표시
cv2.circle(img, (100, 50), 5, (255, 0, 0), -1)
cv2.circle(img, (200, 50), 5, (0, 255, 0), -1)
cv2.circle(img, (100, 200), 5, (0, 0, 255), -1)

# 짝지은 3개의 좌표로 변환행렬 "계산"
mtrx = cv2.getAffineTransform(pts1, pts2)
# affine 변환 적용
dst = cv2.warpAffine(img, mtrx, (int(cols*1.5), rows))

cv2.imshow('original', img)
cv2.imshow('affine', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 5.2.2 원근 변환
- 4개의 점을 짝 지어 매핑할 수 있다면, 원근변환에 필요한 3x3 변환행렬 계산 가능한 cv2.getPerspectiveTransform() 함수를 제공
- matrix = cv2.getPerspectiveTransform(pts1, pts2)
    - pts1 : 변환 전 좌표 4개 (numpy 4x2, float32)
    - pts2 : 변환 후 좌표 4개 (numpy 4x2, float32)
    - matrix : 변환 행렬 반환, 3x3
- cv2.warpPerspective() 함수로 원근 변환
- cv2.warpPerspective(src, mtrx, dsize, [, dst, flags, borderMode, borderValue]) 사용방법은 동일

In [19]:
# affine transform 행렬
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('img/fish.jpg')
rows, cols = img.shape[0:2]

# 변환 전, 후 각 4개의 좌표 생성
pts1 = np.float32([[0, 0], [0, rows], [cols, 0], [cols, rows]])
pts2 = np.float32([[100, 50], [10, rows-50], [cols-100, 50], [cols-10, rows-50]])

# 변환 전 좌표 이미지에 표시
cv2.circle(img, (0, 0), 10, (255, 0, 0), -1)
cv2.circle(img, (0, rows), 10, (0, 255, 0), -1)
cv2.circle(img, (cols, 0), 10, (0, 0, 255), -1)
cv2.circle(img, (cols, rows), 10, (0, 255, 255), -1)

# 짝지은 3개의 좌표로 변환행렬 "계산"
mtrx = cv2.getPerspectiveTransform(pts1, pts2)
# affine 변환 적용
dst = cv2.warpPerspective(img, mtrx, (cols, rows))

cv2.imshow('original', img)
cv2.imshow('perspective', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
# scan
import cv2
import numpy as np

win_name = 'scanning'
img = cv2.imread('img/paper.jpg')
rows, cols = img.shape[:2]
draw = img.copy()
pts_cnt = 0
pts = np.zeros((4, 2), dtype=np.float32)

def onMouse(event, x, y, flags, param):
    global pts_cnt  # 마우스로 찍은 좌표의 갯수 저장
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)    # 좌표에 초록색 동그라미 표시
        cv2.imshow(win_name, draw)

        pts[pts_cnt] = [x, y]   # 마우스 좌표 저장
        pts_cnt += 1
        if pts_cnt == 4:        # 좌표 4개 수집
            # 좌표 4개 중 상하좌우 찾기 ---② 
            sm = pts.sum(axis=1)                 # 4쌍의 좌표 각각 x+y 계산
            diff = np.diff(pts, axis = 1)       # 4쌍의 좌표 각각 x-y 계산

            topLeft = pts[np.argmin(sm)]         # x+y가 가장 값이 좌상단 좌표
            bottomRight = pts[np.argmax(sm)]     # x+y가 가장 큰 값이 좌상단 좌표
            topRight = pts[np.argmin(diff)]     # x-y가 가장 작은 것이 우상단 좌표
            bottomLeft = pts[np.argmax(diff)]   # x-y가 가장 큰 값이 좌하단 좌표

            # 변환 전 4개 좌표 
            pts1 = np.float32([topLeft, topRight, bottomRight , bottomLeft])

            # 변환 후 영상에 사용할 서류의 폭과 높이 계산
            w1 = abs(bottomRight[0] - bottomLeft[0])    # 상단 좌우 좌표간의 거리
            w2 = abs(topRight[0] - topLeft[0])          # 하당 좌우 좌표간의 거리
            h1 = abs(topRight[1] - bottomRight[1])      # 우측 상하 좌표간의 거리
            h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 상하 좌표간의 거리
            width = int(max([w1, w2]))                       # 두 좌우 거리간의 최대값이 서류의 폭
            height = int(max([h1, h2]))                      # 두 상하 거리간의 최대값이 서류의 높이
            # print(width, height)

            # 변환 후 4개의 좌표
            pts2 = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])

            # 변환 행렬 계산 
            mtrx = cv2.getPerspectiveTransform(pts1, pts2)
            # 원근 변환 적용
            result = cv2.warpPerspective(img, mtrx, (width, height), None, None)
            cv2.imshow('scanned', result)

cv2.imshow(win_name, img)
cv2.setMouseCallback(win_name, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 5.2.3 삼각형 어핀 변환
- 삼각 분할 : 입체적 표현이나 모핑(morphing) 기술에 사용
    * 모핑 기술 : 하나의 물체가 다른 물체로 자연스럽게 변하는 것
    
- 사용을 위해서는 7개의 전처리 과정 필요
    1. 변환 전 삼각형 좌표 3쌍 정하기
    2. 변환 후 삼각형 좌표 3쌍 정하기
    3. 과정 1의 삼각형 완전히 감싸는 외접 사각형 좌표 구하기
    4. 과정 3의 사각형 영역을 관심영역으로 지정
    5. 과정 4의 관심영역을 대상으로 과정 1과 과정 2의 좌표로 변환행렬을 구해 어핀 변환
    6. 과정 5의 변환된 관심영역에서 과정 2의 삼각형 좌표만 마스킹
    7. 과정 6의 마스크를 이용해서 원본 또는 다른 영상에 합성

- x, y, w, h = cv2.boundingRect(pts)
    - pts : 다각형 좌표
    - x, y, w, h : 외접 사각형의 좌표와 폭과 높이

- cv2.fillConvexPoly(img, points, color, [, lineType])
    - img : 입력 영상
    - points : 다각형 꼭짓점 좌표
    - color : 채우기에 사용할 색상
    - lineType : 선그리기 알고리즘 선택 플래그

In [36]:
# triangle affine
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')
draw = img.copy()
img2 = img.copy()

# 변환 전, 후 삼각형 좌표
pts1 = np.float32([[188, 14], [85, 202], [294, 216]])
pts2 = np.float32([[128, 40], [85, 307], [306, 167]])

# 각 삼각형을 완전히 감싸는 사각형 좌표
x1, y1, w1, h1 = cv2.boundingRect(pts1)
x2, y2, w2, h2 = cv2.boundingRect(pts2)

# 사각형 이용한 관심영역 설정
roi1 = img[y1:y1+h1, x1:x1+w1]
roi2 = img2[y2:y2+h2, x2:x2+w2]

# 관심영역을 기준으로 좌표 계산
offset1 = np.zeros((3, 2), dtype=np.float32)
offset2 = np.zeros((3, 2), dtype=np.float32)
for i in range(3):
    offset1[i][0], offset1[i][1] = pts1[i][0] - x1, pts1[i][1] - y1
    offset2[i][0], offset2[i][1] = pts2[i][0] - x2, pts2[i][1] - y2

# 관심영역을 주어진 삼각형 좌표로 어핀 변환
mtrx = cv2.getAffineTransform(offset1, offset2)
warped = cv2.warpAffine(roi1, mtrx, (w2, h2), None, cv2.INTER_LINEAR, cv2.BORDER_REFLECT_101)

# 어핀변환 후 삼각형만 골라내기 위한 마스크 생성
mask = np.zeros((h2, w2), dtype=np.uint8)
cv2.fillConvexPoly(mask, np.int32(offset2), (255))

# 삼각형 영역만 마스킹해서 합성
warped_masked = cv2.bitwise_and(warped, warped, mask=mask)
roi2_masked = cv2.bitwise_and(roi2, roi2, mask=cv2.bitwise_not(mask))
roi2_masked = roi2_masked + warped_masked
img2[y2:y2+h2, x2:x2+w2] = roi2_masked

# result
cv2.rectangle(draw, (x1, y1), (x1+w1, y1+h1), (0, 255, 0), 1)
cv2.polylines(draw, [pts1.astype(np.int32)], True, (255, 0, 0), 1)
cv2.rectangle(img2, (x2, y2), (x2+w2, y2+h2), (0, 255, 0), 1)
cv2.imshow('origin', draw)
cv2.imshow('warped triangle', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()